# Crime Analyzer — General — Feature Engineering

Purpose
- Add temporal, demographic, and POI-derived signals while keeping transformations explainable
- Prepare a modeling-ready dataset for classification and pattern discovery

Inputs
- JupyterOutputs/DataIntegrated/cleaned_integrated_crime_data.csv
- Documents/PDCode_PenalLaw.xlsx (PD code → description mapping)

Outputs
- JupyterOutputs/FeatureEngineered/feature_engineered_crime_data.csv

Design decisions
- Impute OFNS_DESC from PD_CD via official codebook when '(NULL)'
- Temporal extraction from CMPLNT_FR_DT/CMPLNT_FR_TM with explicit format; derived HOUR/WEEKDAY/SEASON/TIME_BUCKET
- Holiday/Payday features via holidays.us and simple heuristics (1st/15th)
- Location enrichment: distance-based imputation of PREM_TYP_DESC when nearest POI ≤ 30m; fallback rules for street crimes
- Consolidate PARKS_NM into PREM_TYP_DESC, then drop original
- Interaction flags SAME_AGE_GROUP and SAME_SEX with robust placeholder handling
- Conservative row dropping step with df.dropna() only once; prefer imputations before that

## Imports

DataFrame transforms (pandas/numpy), feature engineering utilities (sklearn), and calendar libraries for holiday/payday signals. Warnings are silenced for clarity.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import openpyxl
import holidays
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

warnings.filterwarnings('ignore')

## Paths

Inputs from JupyterOutputs/DataIntegrated; outputs to JupyterOutputs/FeatureEngineered. This keeps the pipeline stateless and CI-friendly.

In [ ]:
# Path variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "JupyterOutputs"))
cleaned_integrated_dir = os.path.join(base_dir, "DataIntegrated")
cleaned_integrated_data_file = os.path.join(cleaned_integrated_dir, "cleaned_integrated_crime_data.csv")
feature_engineering_dir = os.path.join(base_dir, "FeatureEngineered")
feature_engineered_file_path = os.path.join(feature_engineering_dir, "feature_engineered_crime_data.csv")
pd_codes_file = os.path.join(os.path.dirname(os.path.dirname(base_dir)), "Documents", "PDCode_PenalLaw.xlsx")

# Check if pd_codes_file exists
if not os.path.exists(pd_codes_file):
    raise FileNotFoundError(f"PD codes file not found at: {pd_codes_file}")

# Ensure output directory exists
os.makedirs(feature_engineering_dir, exist_ok=True)

print(f"Base directory: {base_dir}")
print(f"Feature engineering output directory: {feature_engineering_dir}")
print(f"Looking for cleaned integrated data at: {cleaned_integrated_data_file}")
print(f"Looking for PD codes file at: {pd_codes_file}")

Base directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs
Feature engineering output directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\FeatureEngineered
Looking for cleaned integrated data at: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\DataIntegrated\cleaned_integrated_crime_data.csv
Looking for PD codes file at: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\..\Documents\PDCode_PenalLaw.xlsx


# Load Integrated Data

Load the dataset produced by the Data Integration phase.

In [9]:
# Load dataset
print("=== Loading Integrated Data ===")
try:
    if os.path.exists(cleaned_integrated_data_file):
        df = pd.read_csv(cleaned_integrated_data_file)
        initial_rows = len(df)
        print(f"Dataset loaded successfully: {initial_rows} rows and {df.shape[1]} columns")
        print(f"Columns in the dataset: {df.columns.tolist()}")

        # Basic validation
        if initial_rows == 0:
            raise ValueError("Dataset is empty")
        if df.shape[1] < 5:
            raise ValueError("Dataset has insufficient columns")

    else:
        raise FileNotFoundError(f"Could not find cleaned integrated dataset at: {cleaned_integrated_data_file}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise RuntimeError(f"Failed to load required dataset for feature engineering: {e}")

=== Loading Integrated Data ===
Dataset loaded successfully: 2496759 rows and 34 columns
Columns in the dataset: ['BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE']


## Initial Data Overview

Display basic information, summary statistics, and a sample of the loaded dataset.

In [10]:
# Display basic dataset overview
print("\n=== Dataset Overview ===")
print(df.info())
print("\n=== Summary Statistics ===")
print(df.describe())
print("\n=== Sample Row ===")
print(df.sample())


=== Dataset Overview ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496759 entries, 0 to 2496758
Data columns (total 34 columns):
 #   Column              Dtype  
---  ------              -----  
 0   BORO_NM             object 
 1   CMPLNT_FR_DT        object 
 2   CMPLNT_FR_TM        object 
 3   KY_CD               int64  
 4   LAW_CAT_CD          object 
 5   LOC_OF_OCCUR_DESC   object 
 6   OFNS_DESC           object 
 7   PARKS_NM            object 
 8   PD_CD               int64  
 9   PREM_TYP_DESC       object 
 10  SUSP_AGE_GROUP      object 
 11  SUSP_RACE           object 
 12  SUSP_SEX            object 
 13  VIC_AGE_GROUP       object 
 14  VIC_RACE            object 
 15  VIC_SEX             object 
 16  Latitude            float64
 17  Longitude           float64
 18  BAR_DISTANCE        float64
 19  NIGHTCLUB_DISTANCE  float64
 20  ATM_DISTANCE        float64
 21  ATMS_COUNT          float64
 22  BARS_COUNT          float64
 23  BUS_STOPS_COUNT     float64
 24

# Load External PD Codes

Load the external Excel file containing mappings between PD codes and descriptions.

In [ ]:
print("\n=== Loading External PD Codes ===")
try:
    if os.path.exists(pd_codes_file):
        df_codes = pd.read_excel(pd_codes_file)
        initial_rows = len(df_codes)
        print(f"PD Codes dataset loaded successfully: {initial_rows} rows and {df_codes.shape[1]} columns")
        print(f"Columns in the codes dataset: {df_codes.columns.tolist()}")
    else:
        raise FileNotFoundError(f"Could not find PD codes dataset at: {pd_codes_file}")
except Exception as e:
    print(f"Error loading PD codes dataset: {e}")
    print(f"Error details: {e}")


=== Loading External PD Codes ===
PD Codes dataset loaded successfully: 4671 rows and 5 columns
Columns in the codes dataset: ['PDCODE_VALUE', 'LAW_NYS', 'CATEGORY', 'LIT_LONG', 'LIT_SHORT']


# Feature Engineering

This section focuses on creating new features and refining existing ones based on domain knowledge and data characteristics.

## 1) Impute OFNS_DESC from PD_CD

We rely on PDCode_PenalLaw.xlsx to fill '(NULL)' offense descriptions. Mapping uses the first occurrence per PD code and preserves originals when lookup fails.

In [ ]:
print("\n=== Feature Engineering: Imputing OFNS_DESC ===")
# Ensure both codes are of type string
df['PD_CD'] = df['PD_CD'].astype(str)
df_codes['PDCODE_VALUE'] = df_codes['PDCODE_VALUE'].astype(str)

# Create a mapping dictionary from the first occurrence of each code
code_to_lit_short = df_codes.drop_duplicates(subset='PDCODE_VALUE').set_index('PDCODE_VALUE')['LIT_SHORT'].to_dict()

# Fill OFNS_DESC where it's '(null)' using the mapping
null_mask = df['OFNS_DESC'] == '(null)'
initial_nulls = df['OFNS_DESC'].isin(['(null)', None, np.nan]).sum()
df.loc[null_mask, 'OFNS_DESC'] = (
    df.loc[null_mask, 'PD_CD'].map(code_to_lit_short)
    .str.strip()
    .fillna(df.loc[null_mask, 'OFNS_DESC']) # Keep original if map fails
)

# Count missing after imputation
final_nulls = df['OFNS_DESC'].isin(['(null)', None, np.nan]).sum()

# Print summary
print(f"OFNS_DESC missing before imputation: {initial_nulls}")
print(f"OFNS_DESC missing after imputation:  {final_nulls}")
print(f"Number of values filled: {initial_nulls - final_nulls}")


=== Feature Engineering: Imputing OFNS_DESC ===
OFNS_DESC missing before imputation: 44
OFNS_DESC missing after imputation:  0
Number of values filled: 44


## 2) Temporal features

We build HOUR, DAY, WEEKDAY, IS_WEEKEND, MONTH, YEAR, SEASON, and TIME_BUCKET from CMPLNT_FR_DT/CMPLNT_FR_TM using the explicit format '%Y/%m/%d %H:%M:%S.%f' and safely warn on NaT.

In [ ]:
print("\n=== Feature Engineering: Temporal Features ===")
# Unified timestamp (temporary)
df['TIMESTAMP'] = pd.to_datetime(
    df['CMPLNT_FR_DT'] + ' ' + df['CMPLNT_FR_TM'],
    format='%Y/%m/%d %H:%M:%S.%f', # Adjusted format based on previous steps
    errors='coerce'
)

# Handle potential coercion errors (if any)
invalid_timestamps = df['TIMESTAMP'].isna().sum()
if invalid_timestamps > 0:
    print(f"Warning: {invalid_timestamps} rows had invalid date/time formats and resulted in NaT timestamps.")

# Hour of the day
df['HOUR'] = df['TIMESTAMP'].dt.hour

# Day
df['DAY'] = df['TIMESTAMP'].dt.day

# Weekday
df['WEEKDAY'] = df['TIMESTAMP'].dt.day_name().str.upper()

# Flag weekend (1 = Saturday/Sunday)
df['IS_WEEKEND'] = df['WEEKDAY'].isin(['SATURDAY', 'SUNDAY']).astype(int)

# Month
df['MONTH'] = df['TIMESTAMP'].dt.month

# Year
df['YEAR'] = df['TIMESTAMP'].dt.year

# Season
def map_season(month):
    if pd.isna(month): return 'UNKNOWN' # Handle potential NaNs from NaT timestamps
    if month in [12, 1, 2]: return 'WINTER'
    elif month in [3, 4, 5]: return 'SPRING'
    elif month in [6, 7, 8]: return 'SUMMER'
    else: return 'AUTUMN'
df['SEASON'] = df['MONTH'].apply(map_season)

# Time Bucket
def time_bucket(hour):
    if pd.isna(hour): return 'UNKNOWN' # Handle potential NaNs
    if hour < 6: return 'NIGHT'
    elif hour < 12: return 'MORNING'
    elif hour < 18: return 'AFTERNOON'
    else: return 'EVENING'
df['TIME_BUCKET'] = df['HOUR'].apply(time_bucket)

# Drop intermediate and original time columns
df = df.drop(columns=['CMPLNT_FR_TM', 'CMPLNT_FR_DT', 'TIMESTAMP'])
print("Created temporal features: HOUR, DAY, WEEKDAY, IS_WEEKEND, MONTH, YEAR, SEASON, TIME_BUCKET")
print(f"Columns after adding temporal features: {df.columns.tolist()}")


=== Feature Engineering: Temporal Features ===
Created temporal features: HOUR, DAY, WEEKDAY, IS_WEEKEND, MONTH, YEAR, SEASON, TIME_BUCKET
Columns after adding temporal features: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET']


### 2.1 Holiday and payday

IS_HOLIDAY uses holidays.US by year; IS_PAYDAY flags 1st and 15th. These are heuristics to capture coarse pay-cycle effects.

In [ ]:
print("\n=== Feature Engineering: Holiday and Payday Features ===")
# Recreate a datetime series from YEAR, MONTH, DAY for holiday and payday checking
# Ensure YEAR, MONTH, DAY are present and are not NaN before attempting conversion
if 'YEAR' in df.columns and 'MONTH' in df.columns and 'DAY' in df.columns:
    # Create a temporary date column, handling potential errors by setting to NaT
    # This is crucial if YEAR, MONTH, DAY could form invalid dates (e.g., Feb 30)
    date_components = df[['YEAR', 'MONTH', 'DAY']].dropna()
    df_dates = pd.to_datetime(date_components, errors='coerce')

    # Initialize US holidays
    us_holidays = holidays.US(years=df_dates.dt.year.unique())

    # Create IS_HOLIDAY column
    df['IS_HOLIDAY'] = 0
    df.loc[df_dates.index, 'IS_HOLIDAY'] = df_dates.dt.date.apply(lambda date: 1 if date in us_holidays else 0).astype(int)
    print(f"Created IS_HOLIDAY feature. {df['IS_HOLIDAY'].sum()} instances on a holiday.")

    # Create IS_PAYDAY column (assuming 1st and 15th of the month)
    df['IS_PAYDAY'] = 0
    df.loc[df_dates.index, 'IS_PAYDAY'] = df_dates.apply(lambda x: 1 if x.day == 1 or x.day == 15 else 0).astype(int)
    print(f"Created IS_PAYDAY feature. {df['IS_PAYDAY'].sum()} instances on an assumed payday.")
else:
    print("Warning: YEAR, MONTH, or DAY column not found. Skipping IS_HOLIDAY and IS_PAYDAY creation.")

print(f"Columns after adding holiday/payday features: {df.columns.tolist()}")


=== Feature Engineering: Holiday and Payday Features ===
Created IS_HOLIDAY feature. 82010 instances on a holiday.
Created IS_PAYDAY feature. 180576 instances on an assumed payday.
Columns after adding holiday/payday features: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY']


## 3. Refine Location Features

Improve location-related features using imputation and rule-based logic.

## 3) Location features

3.1 PREM_TYP_DESC imputation
- If nearest POI within 30m, assign a consistent premise class (e.g., BAR/NIGHT CLUB, ATM, TRANSIT)
- Else, fallback for street-like offenses (robbery/larceny/etc.) to 'STREET'
- Remaining nulls default to 'OTHER'
Helper columns are dropped after use to keep the schema clean.

In [ ]:
print("\n=== Feature Engineering: Refining Location Features - Impute PREM_TYP_DESC ===")
# Identify rows where PREM_TYP_DESC is missing or placeholder
missing_prem_placeholders = ['(NULL)', None, np.nan, 'UNKNOWN'] # Include UNKNOWN from previous cleaning
mask_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
initial_missing_prem = mask_null_prem.sum()
print(f"Initial missing/placeholder PREM_TYP_DESC: {initial_missing_prem}")

# Compute nearest POI and its distance (helper columns)
poi_cols = ['BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'METRO_DISTANCE']
df['NEAREST_POI_DIST'] = df[poi_cols].min(axis=1)
df['NEAREST_POI_TYPE'] = df[poi_cols].idxmin(axis=1)

# Mapping from distance column name to PREM_TYP_DESC value
distance_to_prem = {
    'BAR_DISTANCE': 'BAR/NIGHT CLUB',
    'NIGHTCLUB_DISTANCE': 'BAR/NIGHT CLUB',
    'ATM_DISTANCE': 'ATM',
    'METRO_DISTANCE': 'TRANSIT - NYC SUBWAY'
}

# Distance-based imputation: only if nearest POI is within 30 meters
mask_impute_by_dist = mask_null_prem & (df['NEAREST_POI_DIST'] <= 30)
df.loc[mask_impute_by_dist, 'PREM_TYP_DESC'] = df.loc[mask_impute_by_dist, 'NEAREST_POI_TYPE'].map(distance_to_prem)
imputed_by_dist_count = mask_impute_by_dist.sum()
print(f"Imputed {imputed_by_dist_count} PREM_TYP_DESC based on POI proximity (<= 30m).")

# Drop helper columns
df.drop(columns=['NEAREST_POI_DIST', 'NEAREST_POI_TYPE'], inplace=True)

# Fallback using OFNS_DESC for street-related crimes
mask_still_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
street_crime_keywords = ['BURGLARY', 'ROBBERY', 'ASSAULT', 'GRAND LARCENY', 'PETIT LARCENY', 'UNAUTHORIZED USE OF A VEHICLE']
mask_impute_by_offense = mask_still_null_prem & df['OFNS_DESC'].str.contains('|'.join(street_crime_keywords), na=False, case=False)
df.loc[mask_impute_by_offense, 'PREM_TYP_DESC'] = 'STREET'
imputed_by_offense_count = mask_impute_by_offense.sum()
print(f"Imputed {imputed_by_offense_count} PREM_TYP_DESC as 'STREET' based on OFNS_DESC.")

# Final default for any remaining missing values
mask_final_null_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders)
final_default_count = mask_final_null_prem.sum()
df.loc[mask_final_null_prem, 'PREM_TYP_DESC'] = 'OTHER'
print(f"Filled remaining {final_default_count} missing PREM_TYP_DESC with 'OTHER'.")

# Final check
final_missing_prem = df['PREM_TYP_DESC'].isin(missing_prem_placeholders).sum()
print(f"PREM_TYP_DESC missing/placeholders after imputation: {final_missing_prem}")


=== Feature Engineering: Refining Location Features - Impute PREM_TYP_DESC ===
Initial missing/placeholder PREM_TYP_DESC: 35590
Imputed 2225 PREM_TYP_DESC based on POI proximity (<= 30m).
Imputed 12816 PREM_TYP_DESC as 'STREET' based on OFNS_DESC.
Filled remaining 20549 missing PREM_TYP_DESC with 'OTHER'.
PREM_TYP_DESC missing/placeholders after imputation: 0


### 3.2 Consolidate park information

Map non-null PARKS_NM to PREM_TYP_DESC = 'PARK/PLAYGROUND', then drop PARKS_NM to avoid leakage and redundancy.

In [16]:
print("\n=== Feature Engineering: Refining Location Features - Consolidate Park Info ===")
# Group PARKS_NM into PREM_TYP_DESC
if 'PARKS_NM' in df.columns:
    park_mask = df['PARKS_NM'].notna() & (df['PARKS_NM'] != '(NULL)') & (df['PARKS_NM'] != 'UNKNOWN')
    park_update_count = park_mask.sum()
    df.loc[park_mask, 'PREM_TYP_DESC'] = 'PARK/PLAYGROUND'
    print(f"Updated {park_update_count} PREM_TYP_DESC entries to 'PARK/PLAYGROUND' based on PARKS_NM.")
    # Drop the original PARKS_NM column as it's now consolidated
    df.drop(columns=['PARKS_NM'], inplace=True)
    print("Dropped PARKS_NM column.")
else:
    print("PARKS_NM column not found, skipping consolidation.")


=== Feature Engineering: Refining Location Features - Consolidate Park Info ===
Updated 15739 PREM_TYP_DESC entries to 'PARK/PLAYGROUND' based on PARKS_NM.
Dropped PARKS_NM column.


### 3.3 LOC_OF_OCCUR_DESC imputation

Rule-based mapping from PREM_TYP_DESC to INSIDE/REAR/UNKNOWN; refine with OFNS_DESC for inside-prone offenses; default remaining to UNKNOWN.

In [ ]:
print("\n=== Feature Engineering: Refining Location Features - Impute LOC_OF_OCCUR_DESC ===")
# Identify missing/placeholder values
missing_loc_placeholders = ['(NULL)', None, np.nan, 'UNKNOWN']
mask_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
initial_missing_loc = mask_null_loc.sum()
print(f"Initial missing/placeholder LOC_OF_OCCUR_DESC: {initial_missing_loc}")

# Rule-based mapping from PREM_TYP_DESC to LOC_OF_OCCUR_DESC
prem_to_loc = {
    # indoor locations
    'GROCERY/BODEGA':                     'INSIDE',
    'RESIDENCE - APT. HOUSE':             'INSIDE',
    'RESIDENCE-HOUSE':                    'INSIDE',
    'RESIDENCE - PUBLIC HOUSING':         'INSIDE',
    'DEPARTMENT STORE':                   'INSIDE',
    'CHAIN STORE':                        'INSIDE',
    'DRUG STORE':                         'INSIDE',
    'FOOD SUPERMARKET':                   'INSIDE',
    'COMMERCIAL BUILDING':                'INSIDE',
    'BANK':                               'INSIDE',
    'PUBLIC BUILDING':                    'INSIDE',
    'HOTEL/MOTEL':                        'INSIDE',
    'HOMELESS SHELTER':                   'INSIDE',
    'PUBLIC SCHOOL':                      'INSIDE',
    'PRIVATE/PAROCHIAL SCHOOL':           'INSIDE',
    'COLLEGE/UNIVERSITY':                 'INSIDE',
    'HOSPITAL':                           'INSIDE',
    'DOCTOR/DENTIST OFFICE':              'INSIDE',
    'GYM/FITNESS FACILITY':               'INSIDE',
    'BAR/NIGHT CLUB':                     'INSIDE',
    'RESTAURANT/DINER':                   'INSIDE',
    'FAST FOOD':                          'INSIDE',
    'DRY CLEANER/LAUNDRY':                'INSIDE',
    'BEAUTY & NAIL SALON':                'INSIDE',
    'CLOTHING/BOUTIQUE':                  'INSIDE',
    'JEWELRY':                            'INSIDE',
    'PHOTO/COPY':                         'INSIDE',
    'VIDEO STORE':                        'INSIDE',
    'STORE UNCLASSIFIED':                 'INSIDE',
    'SMALL MERCHANT':                     'INSIDE',
    'CANDY STORE':                        'INSIDE',
    'VARIETY STORE':                      'INSIDE',
    'SHOE':                               'INSIDE',
    'CHECK CASHING BUSINESS':             'INSIDE',
    'STORAGE FACILITY':                   'INSIDE',
    'REAL ESTATE':                        'INSIDE',
    'SOCIAL CLUB/POLICY':                 'INSIDE',
    'OTHER HOUSE OF WORSHIP':             'INSIDE',
    'CHURCH':                             'INSIDE',
    'SYNAGOGUE':                          'INSIDE',
    'MOSQUE':                             'INSIDE',
    'DAYCARE FACILITY':                   'INSIDE',
    'ABANDONED BUILDING':                 'INSIDE',
    'LOAN COMPANY':                       'INSIDE',
    'TAXI (YELLOW LICENSED)':             'INSIDE',
    'TAXI (LIVERY LICENSED)':             'INSIDE',
    'TAXI/LIVERY (UNLICENSED)':           'INSIDE',
    'BUS (NYC TRANSIT)':                  'INSIDE',
    'BUS (OTHER)':                        'INSIDE',
    'TRANSIT FACILITY (OTHER)':           'INSIDE',
    'AIRPORT TERMINAL':                   'INSIDE',
    'FERRY/FERRY TERMINAL':               'INSIDE',

    # Outdoor or semi‐outdoor locations
    'STREET':                             'REAR',
    'HIGHWAY/PARKWAY':                    'REAR',
    'TUNNEL':                             'REAR',
    'PARK/PLAYGROUND':                    'REAR',
    'OPEN AREAS (OPEN LOTS)':             'REAR',
    'PARKING LOT/GARAGE (PUBLIC)':        'REAR',
    'PARKING LOT/GARAGE (PRIVATE)':       'REAR',
    'GAS STATION':                        'REAR',
    'ATM':                                'REAR',
    'BUS STOP':                           'REAR',
    'BUS TERMINAL':                       'REAR',
    'TRAMWAY':                            'REAR',
    'BRIDGE':                             'REAR',
    'MARINA/PIER':                        'REAR',
    'CEMETERY':                           'REAR',
    'CONSTRUCTION SITE':                  'REAR',
    'MOBILE FOOD':                        'REAR',
    'FERRY/FERRY TERMINAL':               'REAR',  # Also treated as terminal interior above

    # Catch-all for others
    'OTHER':                              'UNKNOWN',
    'STORE UNCLASSIFIED':                 'UNKNOWN',
    'TRANSIT - NYC SUBWAY':               'UNKNOWN' # Can be inside station, on platform, or on train
}

# Apply the mapping to fill missing LOC_OF_OCCUR_DESC
df.loc[mask_null_loc, 'LOC_OF_OCCUR_DESC'] = df.loc[mask_null_loc, 'PREM_TYP_DESC'].map(prem_to_loc)
imputed_by_prem_count = mask_null_loc.sum() - df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders).sum()
print(f"Imputed {imputed_by_prem_count} LOC_OF_OCCUR_DESC based on PREM_TYP_DESC mapping.")

# Refinement using OFNS_DESC for potential inside crimes
mask_still_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
inside_crime_keywords = ['BURGLARY', 'TRESPASS', 'ROBBERY', 'ASSAULT'] # Keywords suggesting inside location
mask_impute_by_offense_loc = mask_still_null_loc & df['OFNS_DESC'].str.contains('|'.join(inside_crime_keywords), na=False, case=False)
df.loc[mask_impute_by_offense_loc, 'LOC_OF_OCCUR_DESC'] = 'INSIDE'
imputed_by_offense_loc_count = mask_impute_by_offense_loc.sum()
print(f"Imputed {imputed_by_offense_loc_count} LOC_OF_OCCUR_DESC as 'INSIDE' based on OFNS_DESC.")

# Final default for any remaining missing values
mask_final_null_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders)
final_default_loc_count = mask_final_null_loc.sum()
df.loc[mask_final_null_loc, 'LOC_OF_OCCUR_DESC'] = 'UNKNOWN'
print(f"Filled remaining {final_default_loc_count} missing LOC_OF_OCCUR_DESC with 'UNKNOWN'.")

# Final check
final_missing_loc = df['LOC_OF_OCCUR_DESC'].isin(missing_loc_placeholders).sum()
print(f"LOC_OF_OCCUR_DESC missing/placeholders after imputation: {final_missing_loc}")


=== Feature Engineering: Refining Location Features - Impute LOC_OF_OCCUR_DESC ===
Initial missing/placeholder LOC_OF_OCCUR_DESC: 464374
Imputed 378151 LOC_OF_OCCUR_DESC based on PREM_TYP_DESC mapping.
Imputed 15240 LOC_OF_OCCUR_DESC as 'INSIDE' based on OFNS_DESC.
Filled remaining 70983 missing LOC_OF_OCCUR_DESC with 'UNKNOWN'.
LOC_OF_OCCUR_DESC missing/placeholders after imputation: 70983


## 4) Demographic interactions

Create SAME_AGE_GROUP and SAME_SEX only when both fields are non-placeholder; treat 'U'/'UNKNOWN' as missing for the purpose of matching.

In [ ]:
print("\n=== Feature Engineering: Demographic Interaction Features ===")
# SAME_AGE_GROUP: 1 if suspect and victim share the same age group, else 0
df['SAME_AGE_GROUP'] = 0
# Define valid (non-null, non-placeholder) age-group rows
valid_age_placeholders = ['UNKNOWN', '(NULL)', None, np.nan]
age_valid = (~df['SUSP_AGE_GROUP'].isin(valid_age_placeholders)) & (~df['VIC_AGE_GROUP'].isin(valid_age_placeholders))
# Set flag where age groups match
df.loc[age_valid & (df['SUSP_AGE_GROUP'] == df['VIC_AGE_GROUP']), 'SAME_AGE_GROUP'] = 1
print(f"Created SAME_AGE_GROUP feature. {df['SAME_AGE_GROUP'].sum()} instances where age groups match.")

# SAME_SEX: 1 if suspect and victim have the same sex, else 0
df['SAME_SEX'] = 0
# Define valid (non-null, non-placeholder) sex rows
valid_sex_placeholders = ['U', 'UNKNOWN', '(NULL)', None, np.nan] # 'U' often means Unknown
sex_valid = (~df['SUSP_SEX'].isin(valid_sex_placeholders)) & (~df['VIC_SEX'].isin(valid_sex_placeholders))
# Set flag where sexes match
df.loc[sex_valid & (df['SUSP_SEX'] == df['VIC_SEX']), 'SAME_SEX'] = 1
print(f"Created SAME_SEX feature. {df['SAME_SEX'].sum()} instances where sexes match.")


=== Feature Engineering: Demographic Interaction Features ===
Created SAME_AGE_GROUP feature. 388596 instances where age groups match.
Created SAME_SEX feature. 515461 instances where sexes match.


## 5) Citizen-check relevance flag

Binary TO_CHECK_CITIZENS marks offenses where proactive citizen checks may apply, based on a curated list of offense descriptions.

In [19]:
print("\n=== Feature Engineering: TO_CHECK_CITIZENS Feature ===")
# List of offense descriptions that trigger the flag
to_check_citizens_list = [
    'CRIMINAL TRESPASS', 'CRIMINAL MISCHIEF & RELATED OF', 'HARRASSMENT 2',
    'GRAND LARCENY OF MOTOR VEHICLE', 'PETIT LARCENY', 'GRAND LARCENY',
    'INTOXICATED & IMPAIRED DRIVING', 'FRAUDS', 'THEFT-FRAUD',
    'OFF. AGNST PUB ORD SENSBLTY &', 'BURGLARY', 'ROBBERY', 'FELONY ASSAULT',
    'ASSAULT 3 & RELATED OFFENSES', 'DANGEROUS DRUGS', 'RAPE', 'SEX CRIMES',
    'DANGEROUS WEAPONS', 'ARSON', 'POSSESSION OF STOLEN PROPERTY',
    'UNAUTHORIZED USE OF A VEHICLE', 'FORGERY', 'ENDAN WELFARE INCOMP',
    'OTHER OFFENSES RELATED TO THEF', 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
    'KIDNAPPING & RELATED OFFENSES', 'OTHER OFFENSES RELATED TO THEFT', # Duplicate, keep one
    'OFFENSES RELATED TO CHILDREN', 'BURGLAR\S TOOLS', 'ESCAPE 3',
    'CANNABIS RELATED OFFENSES', 'PETIT LARCENY OF MOTOR VEHICLE',
    'FELONY SEX CRIMES', 'PROSTITUTION & RELATED OFFENSES', 'JOSTLING',
    'KIDNAPPING', 'DISORDERLY CONDUCT', 'INTOXICATED/IMPAIRED DRIVING',
    'DISRUPTION OF A RELIGIOUS SERV', 'FRAUDULENT ACCOSTING', 'THEFT OF SERVICES',
    'UNRSNBLE NOISE', 'OTHER TRAFFIC INFRACTION', 'LOITERING/GAMBLING (CARDS, DIC',
    'UNLAWFUL POSS. WEAP. ON SCHOOL', 'FORTUNE TELLING', 'LOITERING',
    'FAIL RPT WOUNDS'
]
# Remove duplicates from the list
to_check_citizens_list = list(dict.fromkeys(to_check_citizens_list))

# Create the boolean column based on the condition
df['TO_CHECK_CITIZENS'] = df['OFNS_DESC'].isin(to_check_citizens_list).astype(int)
print(f"Created TO_CHECK_CITIZENS feature. {df['TO_CHECK_CITIZENS'].sum()} instances flagged.")


=== Feature Engineering: TO_CHECK_CITIZENS Feature ===
Created TO_CHECK_CITIZENS feature. 2270901 instances flagged.


## 6) Drop remaining NaNs (last resort)

After targeted imputations, we drop remaining NaNs once to stabilize modeling. Consider revisiting this if recall on rare classes is critical.

In [20]:
print("\n=== Dropping Rows with NaN values ===")
initial_rows = len(df)
print(f"Initial number of rows: {initial_rows}")

# Drop rows that contain any NaN value
df_cleaned = df.dropna()

final_rows = len(df_cleaned)
rows_dropped = initial_rows - final_rows

print(f"Number of rows after dropping NaNs: {final_rows}")
print(f"Number of rows dropped: {rows_dropped}")

# Update the main DataFrame reference to the cleaned one
df = df_cleaned

print(f"Current DataFrame shape: {df.shape}")


=== Dropping Rows with NaN values ===
Initial number of rows: 2496759
Number of rows after dropping NaNs: 2496742
Number of rows dropped: 17
Current DataFrame shape: (2496742, 44)


## Save Feature Engineered Dataset

Save the feature engineered dataset before scaling or dimensionality reduction.

In [21]:
print("\n=== Saving Feature Engineered Dataset (Before Scaling/PCA) ===")
# Ensure feature_engineering_dir and feature_engineered_file_path are defined

if 'df' in globals() and 'feature_engineered_file_path' in globals() and 'os' in globals():
    try:
        df.to_csv(feature_engineered_file_path, index=False)
        print(f"Feature engineered dataset (before scaling/PCA) saved successfully to: {feature_engineered_file_path}")
        print(f"Dataset shape: {df.shape}")
        print(f"Columns in the dataset: {df.columns.tolist()}")
    except NameError as ne:
        print(f"A NameError occurred while trying to save: {ne}. Ensure 'df' and path variables are correctly defined.")
    except Exception as e:
        print(f"Error saving feature engineered dataset (before scaling/PCA): {e}")
else:
    print("Error: 'df' or 'feature_engineered_file_path' or 'os' not found in globals. Cannot save the intermediate DataFrame.")


=== Saving Feature Engineered Dataset (Before Scaling/PCA) ===
Feature engineered dataset (before scaling/PCA) saved successfully to: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\FeatureEngineered\feature_engineered_crime_data.csv
Dataset shape: (2496742, 44)
Columns in the dataset: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY', 'SAME_AGE_GROUP', 'SAME_SEX', 'TO_CHECK_CITIZENS']


## Appendix: Feature classification and encoding

This table summarizes feature types and suggested encodings before modeling. Use it as a guide for pipeline configuration.

| Feature | Type | Notes | Suggested encoding |
|---|---|---|---|
| BORO_NM | Categorical (nominal) | Borough name | One-hot |
| KY_CD | Categorical (nominal) | Crime classification code | One-hot or target encoding (if many levels) |
| LAW_CAT_CD | Categorical (ordinal) | FELONY > MISDEMEANOR > VIOLATION | Ordinal mapping |
| LOC_OF_OCCUR_DESC | Categorical (nominal) | INSIDE/REAR/UNKNOWN | One-hot |
| OFNS_DESC | Categorical (nominal, high-card.) | Offense description | Target/impact encoding or top-k one-hot |
| PD_CD | Categorical (nominal, high-card.) | Police code | Target/impact encoding or hashed |
| PREM_TYP_DESC | Categorical (nominal) | Premises type | One-hot |
| SUSP_AGE_GROUP | Categorical (ordinal) | <18, 18-24, 25-44, 45-64, 65+, UNKNOWN | Ordinal mapping |
| SUSP_RACE | Categorical (nominal) | — | One-hot |
| SUSP_SEX | Categorical (nominal) | M/F/U | One-hot |
| VIC_AGE_GROUP | Categorical (ordinal) | As above | Ordinal mapping |
| VIC_RACE | Categorical (nominal) | — | One-hot |
| VIC_SEX | Categorical (nominal) | M/F/U | One-hot |
| Latitude | Numeric (continuous) | NYC bounds filtering applied | Leave or scale |
| Longitude | Numeric (continuous) | NYC bounds filtering applied | Leave or scale |
| BAR_DISTANCE, NIGHTCLUB_DISTANCE, ATM_DISTANCE, METRO_DISTANCE | Numeric (continuous) | Meters to nearest POI | Scale |
| ATMS_COUNT, BARS_COUNT, BUS_STOPS_COUNT, METROS_COUNT, NIGHTCLUBS_COUNT, SCHOOLS_COUNT | Numeric (discrete) | Grid 100m counts | Leave or scale |
| HOUR, DAY, MONTH | Numeric (cyclical) | Time components | Sine/cosine encoding |
| WEEKDAY | Categorical (cyclical) | MON–SUN | One-hot or sine/cosine |
| IS_WEEKEND, IS_HOLIDAY, IS_PAYDAY | Binary | 0/1 flags | As-is |
| YEAR | Numeric (discrete) | Year of event | Leave or one-hot if few years |
| SAME_AGE_GROUP, SAME_SEX | Binary | Interaction flags | As-is |
| TO_CHECK_CITIZENS | Binary | Curated flag | As-is |
| TOTAL_POI_COUNT, POI_DIVERSITY, POI_DENSITY_SCORE | Numeric | Aggregates over POI counts | Scale |
| MIN_POI_DISTANCE, AVG_POI_DISTANCE, MAX_POI_DISTANCE | Numeric | Aggregates over POI distances | Scale |

Tip: For tree-based models, raw categorical encoding (one-hot/ordinal) is fine; for linear models, prefer scaling and impact encoding for high-cardinality variables.